In [1]:
%run 'read_and_process_data.ipynb'

In [2]:
from xgboost.sklearn import XGBClassifier
clf = XGBClassifier()
clf

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [3]:
clf = clf.fit(X, y)
train_yhat = clf.predict(X)
accuracy_score(y, train_yhat)

/usr/lib64/python2.7/site-packages/sklearn/preprocessing/label.py:95: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/lib64/python2.7/site-packages/sklearn/preprocessing/label.py:128: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8412921348314607

In [4]:
dev_yhat = clf.predict(dev_X)
accuracy_score(dev_y, dev_yhat)

0.78770949720670391

In [10]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import numpy as np

space = {
    'max_depth': hp.quniform('max_depth', 3, 10, 2),
    'n_estimators': hp.quniform('n_estimators', 100, 300, 50),
    'gamma': hp.quniform('gamma', 0, 5, 1),
#     'subsample': hp.uniform('subsample', 0.5, 1, 0.1),
#     'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1, 0.1),
#     'learning_rate': hp.loguniform('learning_rate', 0.01, 0.2),
}

# reg_lambda - L2 regularization

def score(params):
    # convert all the numbers to integer
    print(params)
    params = {x: int(y) for x, y in params.iteritems()}
    params['n_jobs'] = -1
    clf = XGBClassifier(**params)
    return {'loss': 1-check_accuracy(clf, X, y, dev_X, dev_y), 'status': STATUS_OK}

trials = Trials()
# best = fmin(score, space, algo=tpe.suggest, trials=trials, max_evals=200)
best = {'gamma': 0.0, 'max_depth': 6.0, 'n_estimators': 200.0}
best = {x: int(y) for x, y in best.iteritems()}
best

{'gamma': 0, 'max_depth': 6, 'n_estimators': 200}

In [11]:
clf_tuned = XGBClassifier(**best)
clf_tuned = clf_tuned.fit(X, y)
train_p = clf_tuned.predict(X)
accuracy_score(y, train_p)

0.8595505617977528

In [12]:
dev_p = clf_tuned.predict(dev_X)
accuracy_score(dev_y, dev_p)

0.83798882681564246

In [13]:
pred = clf_tuned.predict(test_X)

outfile = os.path.join(data_dir, 'prediction.csv')
df = pd.DataFrame()
df['PassengerId'] = passenger_id
df['Survived'] = pred
df.to_csv(outfile, index=False)